# Rede Convolucional e Transfer Learning

### Trabalho 2 da disciplina Noções de Inteligência Artificial - 2/2024
### Alunos: Felipe Lopes Gibin Duarte e Matheus das Neves Fernandes

## Introdução
Neste trabalho usaremos uma rede convolucional pré-treinada (VGG) e a aplicaremos em um problema novo. Experimentaremos com a divisão da base em treinamento, validação e teste, e usaremos validação para o "early stopping" na tentativa de controlar o sobre-ajuste. A base de dados usada é a "TensorFlow Flowers Dataset". Ela contém 3670 imagens coloridas de flores pertencentes a uma de 5 classes: Margarida,
Dente-de-leão, Rosa, Girassol e Tulipa.

## 1. Preparação do Ambiente
Nesta seção, importaremos as bibliotecas necessárias e carregaremos a base de dados.

### 1.1 Importação de bibliotecas 

In [4]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

## 1.2 Carregamento dos dados

In [5]:
## Loading images and labels
(train_ds, train_labels), (test_ds, test_labels) = tfds.load(
"tf_flowers",
split=["train[:70%]", "train[70%:]"], ## Train test split
batch_size=-1,
as_supervised=True, # Include labels
)

## Resizing images
train_ds = tf.image.resize(train_ds, (150, 150))
test_ds = tf.image.resize(test_ds, (150, 150))

print (train_ds.shape)
print (test_ds.shape)

(2569, 150, 150, 3)
(1101, 150, 150, 3)


## 2. Treinando um MLP
Vamos comprovar que um MLP não apresenta desempenho satisfatório para o problema. Vamos evitar o overfitting ao usar a técnica de parada prematura de treinamento (early stopping), após 5 épocas sem melhora no parâmetro monitorado (neste caso a precisão de treinamento, val_accuracy), o treinamento é interrompido. Observamos que o MLP apresenta uma acurácia de cerca de 30%. Conforme esperado ele não resolve bem o problema de classificação.

In [3]:
model_MLP = tf.keras.Sequential([ 
    tf.keras.layers.Flatten(),  
    tf.keras.layers.Dense(256, activation='relu'),   #Camada escondida 1
    tf.keras.layers.Dense(256, activation='relu'),   #Camada escondida 2
    tf.keras.layers.Dense(5, activation='softmax')  #Camada de saída
])   

model_MLP.compile(
    optimizer=tf.keras.optimizers.Adam(),   #Otimizador Adam
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),  #Entropia cruzada
    metrics=['accuracy']
)

es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5, restore_best_weights=True)

model_MLP.fit(
    train_ds, 
    train_labels, 
    epochs=20, 
    validation_split=0.2, 
    batch_size=32, 
    callbacks=[es]
)

# Evaluate the model on the test dataset
loss, accuracy = model_MLP.evaluate(test_ds, test_labels)

# Print the accuracy
print('Accuracy:', accuracy)

Epoch 1/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 10s 127ms/step - accuracy: 0.2551 - loss: 839.1895 - val_accuracy: 0.3288 - val_loss: 203.3070
Epoch 2/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 8s 123ms/step - accuracy: 0.3397 - loss: 120.3524 - val_accuracy: 0.3191 - val_loss: 100.4006
Epoch 3/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 8s 123ms/step - accuracy: 0.3711 - loss: 52.3478 - val_accuracy: 0.3249 - val_loss: 49.7330
Epoch 4/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 8s 124ms/step - accuracy: 0.3389 - loss: 24.4813 - val_accuracy: 0.2549 - val_loss: 15.7129
Epoch 5/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 8s 124ms/step - accuracy: 0.2857 - loss: 9.3323 - val_accuracy: 0.2529 - val_loss: 2.9597
Epoch 6/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 8s 124ms/step - accuracy: 0.2670 - loss: 1.9715 - val_accuracy: 0.2412 - val_loss: 2.1306
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.3015 - loss: 743.2046
Accuracy: 0.30245232582092285


## 3. Uso da rede VGG16 pré treinada
Vamos utilizar a rede VGG16 como pré processamento fixo das imagens. Ainda que essa rede tenha sido treinada em um conjunto de dados muito mais amplo do que somente flores, vamos verificar que podemos reusar esse treinamento para aumentar a performance do nosso problema de classificação.

### 3.1 Processamento da rede VGG
Carregamos o modelo VGG, pre processamos os dados e checamos se o formato deles está em tensores condizentes com o MLP

In [8]:
# Carregando modelo VGG16
base_model = VGG16(weights="imagenet", include_top=False, input_shape=train_ds[0].shape)
base_model.trainable = False ## Not trainable weights

# Dados são processados de maneira semelhante ao treinamento original da VGG16.
train_ds = preprocess_input(train_ds)
test_ds = preprocess_input(test_ds)

# Checando se o formato dos dados esta consistente com o MLP
print(train_ds.shape)
print(test_ds.shape)

# Resumo da arquitetura do modelo
base_model.summary()

(2569, 150, 150, 3)
(1101, 150, 150, 3)


Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 150, 150, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 150, 150, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 150, 150, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 75, 75, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 75, 75, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 75, 75, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 37, 37, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 37, 37, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 37, 37, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 37, 37, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 18, 18, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 18, 18, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 18, 18, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 18, 18, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 9, 9, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 9, 9, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 9, 9, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 9, 9, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 4, 4, 512)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 14,714,688 (56.13 MB)

### 3.2 Treinamento do MLP que sucede a VGG
Ele ta overfitting, usar regularização. 

In [9]:
model_MLP_from_VGG = tf.keras.Sequential([ 
    base_model,                                     #Base do VGG
    tf.keras.layers.Flatten(),  
    tf.keras.layers.Dense(64, activation='relu'),   #Camada escondida 1
    tf.keras.layers.Dense(20, activation='relu'),   #Camada escondida 2
    tf.keras.layers.Dense(5, activation='softmax')  #Camada de saída
])   

model_MLP_from_VGG.compile(
    optimizer=tf.keras.optimizers.Adam(),   #Otimizador Adam
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),  #Entropia cruzada
    metrics=['accuracy']
)

model_MLP_from_VGG.fit(
    train_ds, 
    train_labels, 
    epochs=10, 
    validation_split=0.2, 
    batch_size=32, 
)

# Evaluate the model on the test dataset
loss, accuracy = model_MLP_from_VGG.evaluate(test_ds, test_labels)

# Print the accuracy
print('Accuracy:', accuracy)

Epoch 1/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 147s 2s/step - accuracy: 0.2822 - loss: 4.5851 - val_accuracy: 0.2743 - val_loss: 1.5211
Epoch 2/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 145s 2s/step - accuracy: 0.3976 - loss: 1.4347 - val_accuracy: 0.4066 - val_loss: 1.3972
Epoch 3/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 145s 2s/step - accuracy: 0.5392 - loss: 1.2361 - val_accuracy: 0.4300 - val_loss: 1.3648
Epoch 4/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 145s 2s/step - accuracy: 0.5518 - loss: 1.0627 - val_accuracy: 0.4572 - val_loss: 1.2907
Epoch 5/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 150s 2s/step - accuracy: 0.6105 - loss: 0.9339 - val_accuracy: 0.5837 - val_loss: 1.1145
Epoch 6/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 167s 3s/step - accuracy: 0.6862 - loss: 0.8995 - val_accuracy: 0.5817 - val_loss: 1.2810
Epoch 7/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 162s 2s/step - accuracy: 0.6895 - loss: 0.7428 - val_accuracy: 0.6323 - val_loss: 1.2598
Epoch 8/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 166s 3s/step - accuracy: 0.7496 - loss: 0.6275 - val_accuracy: 0.6284 - v